# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## **🗒️ In this notebook we will see how to create a training dataset from the feature groups:** 
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset** for training fraud detection model.

![tutorial-flow](../images/02_training-dataset.png) 

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/164


Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🔪 Feature Selection </span>

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
trans_fg = fs.get_feature_group('transactions_online_fg', version=1)

# Select features for training data.
ds_query = trans_fg.select(["fraud_label", "loc_delta_t_plus_1", "loc_delta_t_minus_1", "time_delta_t_plus_1", "time_delta_t_minus_1", "country"])

# uncomment this if you would like to view query results
# ds_query.show(5)

Recall that you computed the features in `transactions_fg`. If you had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join you would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

🤖 Transformation Functions </span>

We will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [3]:
# Load the transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformation functions.
transformation_functions = {
    "loc_delta_t_plus_1": min_max_scaler, 
    "loc_delta_t_minus_1": min_max_scaler, 
    "time_delta_t_plus_1": min_max_scaler, 
    "time_delta_t_minus_1": min_max_scaler,
    "country": label_encoder
}

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.
In order to create a Feature View we may use `fs.create_feature_view()`

In [4]:
feature_view = fs.create_feature_view(
    name='fraud_online_model_view',
    query=ds_query,
    labels=["fraud_label"],
    transformation_functions=transformation_functions
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/164/fs/106/fv/fraud_online_model_view/version/1


To view and explore data in the feature view we can retrieve batch data using `get_batch_data()` method 

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_training_dataset()` method.

**From feature view APIs we can also create training datasts based on even time filters specifing `start_time` and `end_time`** 



In [5]:
from datetime import datetime
date_format = "%Y-%m-%d %H:%M:%S"

In [6]:
# Create training datasets based event time filter
start_time = int(float(datetime.strptime("2022-01-01 00:00:01", date_format).timestamp()) * 1000)
end_time = int(float(datetime.strptime("2022-02-28 23:59:59", date_format).timestamp()) * 1000)

td_jan_feb_version, td_job = feature_view.create_training_data(
        start_time = start_time,
        end_time = end_time,    
        description = 'transactions_dataset_jan_feb',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
    )

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/fraud_online_model_view_1_1_create_fv_td_13072022115741/executions


In [7]:
start_time = int(float(datetime.strptime("2022-03-01 00:00:01", date_format).timestamp()) * 1000)
end_time = int(float(datetime.strptime("2022-03-31 23:59:59", date_format).timestamp()) * 1000)

td_mar_version, td_job = feature_view.create_training_data(
        start_time = start_time,
        end_time = end_time,
        description = 'transactions_dataset_mar',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
 )


Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/fraud_online_model_view_1_2_create_fv_td_13072022115847/executions


## <span style="color:#ff5f27;"> 🪝 Training Dataset retreival </span>

To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset_splits` or `get_training_dataset` methods. If version is not provided or provided version has not already existed, it creates a new version of training data according to given arguments and returns a dataframe. If version is provided and has already existed, it reads training data from storage or feature groups and returns a dataframe. If split is provided, it reads the specific split.

In [8]:
train_jan_feb_x, train_jan_feb_y = feature_view.get_training_data(td_jan_feb_version)

In [9]:
test_mar_x, test_mar_y = feature_view.get_training_data(td_mar_version)

In [10]:
train_jan_feb_x

,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_plus_1,time_delta_t_minus_1,country
0,0.026429,0.105167,0.082340,0.004474,0
1,0.047603,0.208818,0.018629,0.014157,0
2,0.210745,0.156647,0.017069,0.059519,0
3,0.054502,0.055974,0.025656,0.003169,0
4,0.081190,0.080328,0.003342,0.054167,0
...,...,...,...,...,...
272127,0.081158,0.119159,0.025696,0.034686,0
272128,0.002912,0.048349,0.007733,0.049591,0
272129,0.107984,0.037366,0.021496,0.006767,0
272130,0.218193,0.209757,0.063040,0.025142,0


In [11]:
test_mar_x

,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_plus_1,time_delta_t_minus_1,country
0,0.002320,0.119935,0.109213,0.014342,0
1,0.169366,0.167450,0.055270,0.088363,0
2,0.102386,0.173234,0.005226,0.020395,0
3,0.210328,0.050104,0.037844,0.015681,0
4,0.052259,0.109791,0.153961,0.014053,0
...,...,...,...,...,...
85023,0.128743,0.116378,0.018649,0.025914,0
85024,0.045237,0.093660,0.024183,0.024062,0
85025,0.057782,0.090179,0.152458,0.000212,0
85026,0.097188,0.069886,0.062210,0.037553,0


The feature view and training dataset are now visible in the UI

![fg-overview](../images/fv_overview.gif)

### ⛓️ <b> Lineage </b> 
In all the feature groups and feature view you can look at the relation between each abstractions; what feature group created which training dataset and that is used in which model.
This allows for a clear undestanding of the pipeline in relation to each element. 

![provenance](../images/provenance.gif)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the following notebook, you will train a model on the dataset you created in this notebook.